In [130]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

from tqdm import tqdm
import os
import pickle
import uuid
import time

from pathlib import Path

入力情報

In [132]:
""" 対象の基本情報 """


""" navitime関連の情報 """
BusCompanyPage = "https://www.navitime.co.jp/bus/company/00001313/"


DirWriteGtfs = "C:/lab/gtfses/Ishigaki_pj"

In [11]:
chrome_options = Options()
chrome_options.add_argument("--headless")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
"""
当該バス会社の全バス停名とリンクを取得
"""
# 結果格納用辞書 key:nameOfBusStop value:link
nameStop_Lind_dict = {}



for n in ["あ","い","う","え","お","か","き","く","こ","さ","し","す","せ","た","ち","つ","て","と","な","に","は","ひ","ふ","ほ","ま","み","め","や","ろ"]:
    print(n)
    # リンクテキスト名が"{n}"の要素を取得
    driver.get(BusCompanyPage)
    driver.find_element(By.LINK_TEXT, n).click()
    for i in driver.find_element(By.CLASS_NAME,"node-list").find_elements(By.TAG_NAME,"a"):
        nameStop_Lind_dict[i.text] = i.get_attribute("href")
        time.sleep(0.1)
    time.sleep(0.1)

In [98]:
"""
当該バス会社の全バス停名とリンクを取得
"""
# 結果格納用辞書 key:nameOfBusStop value:link
bsInfoAndLinks = []
# 平仮名とリンク
kana_link_dict = {}


# バス停の平仮名からスクレイピング
response = requests.get(BusCompanyPage)
soup = BeautifulSoup(response.text, "html.parser")
links = soup.find_all("dl", class_="kana-frame")

for link in links:
    soup_c = BeautifulSoup(str(link), "html.parser")

    for a in soup_c.select("a"):
        kana_link_dict[a.text] = a["href"]

# バス停名とリンクを取得
for kana_link in list(kana_link_dict.values()):        
    response = requests.get("https://" + kana_link[2:])
    soup = BeautifulSoup(response.text,"html.parser")
    node_contents = soup.find("ul",class_="node-list")
    nodes = BeautifulSoup(str(node_contents),"html.parser").find_all("li")
    for node in nodes:
        soup_c = BeautifulSoup(str(node),"html.parser")
        url = soup_c.find("a")["href"]
        nameBusStop = soup_c.find("rb").text
        id = url.split("/")[-2]
        #結果格納
        bsInfoAndLinks.append({"nameBusStop":nameBusStop,"url":url,"id":id})

In [99]:
bsInfoAndLinks

[{'nameBusStop': '明石(石垣市)',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428229/',
  'id': '00428229'},
 {'nameBusStop': '荒川(石垣市)',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428172/',
  'id': '00428172'},
 {'nameBusStop': '新川公園',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428163/',
  'id': '00428163'},
 {'nameBusStop': '新川五町内',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428222/',
  'id': '00428222'},
 {'nameBusStop': '新川小学校前',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428199/',
  'id': '00428199'},
 {'nameBusStop': '新川団地(沖縄県)',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428215/',
  'id': '00428215'},
 {'nameBusStop': '荒引橋',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428206/',
  'id': '00428206'},
 {'nameBusStop': 'アートホテル石垣島',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428185/',
  'id': '00428185'},
 {'nameBusStop': '石垣空港',
  'url': '//www.navitime.co.jp/bus/diagram/direction/00428191/',
  'i

gtfsフォルダの作成

In [140]:
PathToGdfs = DirWriteGtfs+"/" + str(BusCompanyPage.split("/")[-2])+".gtfs"
Path(PathToGdfs).mkdir(exist_ok=True)

agency.txtの作成

In [141]:
"""
agency_id
"""
agency_id = BusCompanyPage.split("/")[-2]

"""
agency_name
"""
response = requests.get(BusCompanyPage)
agency_name = BeautifulSoup(response.text,"html.parser").find("h1",class_="main-title").text.split(u"\xa0")[0]

"""
agency_url
"""
agency_url = BusCompanyPage

"""
agency_timezone
"""
agency_timezone = "Asia/Tokyo"

"""
agency_lang
"""
agency_lang = "ja"

"""
.txtを作成
"""
pd.DataFrame({"agency_id":[agency_id],"agency_name":[agency_name],"agency_url":[agency_url],"agency_timezone":[agency_timezone],"agency_lang":[agency_lang]}).to_csv(PathToGdfs+"/agency.txt",index=False,sep=",")


stops.txtの作成

In [ ]:
"""
stop_id
"""
agency_id = BusCompanyPage.split("/")[-2]

"""
agency_name
"""
response = requests.get(BusCompanyPage)
agency_name = BeautifulSoup(response.text,"html.parser").find("h1",class_="main-title").text.split(u"\xa0")[0]

"""
agency_url
"""
agency_url = BusCompanyPage

"""
agency_timezone
"""
agency_timezone = "Asia/Tokyo"

"""
agency_lang
"""
agency_lang = "ja"

"""
.txtを作成
"""
pd.DataFrame({"agency_id":[agency_id],"agency_name":[agency_name],"agency_url":[agency_url],"agency_timezone":[agency_timezone],"agency_lang":[agency_lang]}).to_csv(PathToGdfs+"/agency.txt",index=False,sep=",")

In [114]:
import pandas as pd
pd.set_option('display.max_rows', 100)

In [121]:
pd.read_csv("C:/lab/research-master/otp/shimodenbus.gtfs/trips.txt").head(20)

,route_id,service_id,trip_id,trip_headsign,block_id,trip_short_name,direction_id
0,塩生線_B,平日,平日_06時15分_系統E06上,倉敷駅前（倉敷成人病センター前 経由）,NaN,NaN,1.0
1,王子ヶ岳線_A,平日,平日_06時15分_系統I3下,王子ヶ岳登山口,NaN,NaN,0.0
2,天城線_A,平日,平日_06時17分_系統C01上,倉敷駅前,NaN,NaN,1.0
3,茶屋町線_B,平日,平日_06時17分_系統D19上,倉敷駅前（茶屋町駅前 経由）,NaN,NaN,1.0
4,塩生線_B,平日,平日_06時20分_系統E06下,ＪＲ児島駅（倉敷成人病センター前 経由）,NaN,NaN,0.0
5,天城線_A,平日,平日_06時25分_系統C01下,ＪＲ児島駅,NaN,NaN,0.0
6,塩生線_B,平日,平日_06時30分_系統E06上,倉敷駅前（倉敷成人病センター前 経由）,NaN,NaN,1.0
7,興除線_C,平日,平日_06時30分_系統A21上001,天満屋（岡山駅 経由）,NaN,NaN,1.0
8,興除線_B,平日,平日_06時31分_系統A07上,天満屋（汗入 経由）,NaN,NaN,1.0
9,天城線_A,平日,平日_06時40分_系統C01上,倉敷駅前,NaN,NaN,1.0


In [ ]:
# pickleとして辞書を保存
with open(DirTimeSchedule + "/jikokuhyou_link_list(" + str(uuid.uuid1()) + ").pickle", "wb") as f:
    pickle.dump(jikokuhyou_link_list, f)
# read pickle pickle
with open(DirTimeSchedule + "/jikokuhyou_link_list.pickle", "rb") as f:
    targetList = pickle.load(f)

In [ ]:
"""
ダイヤのページから,timeTableへのリンクを取得
"""
dir = "https://www.navitime.co.jp"
diagram_link_list = []
for l in tqdm(targetList):
    html_text = requests.get(l).text
    soup = BeautifulSoup(html_text, 'html.parser')
    for c_dir in soup.find_all("ul", class_="timeTable"):
        time.sleep(0.01)
        diagram_link_list.append(dir+c_dir.find("a").get("href"))

In [ ]:
# pickleとして辞書を保存
with open("C:/research/バス路線データ/scraping/時刻表_単一運用_全リンク.pickle", "wb") as f:
    pickle.dump(diagram_link_list, f)
# read pickle pickle
with open("C:/research/バス路線データ/scraping/時刻表_単一運用_全リンク.pickle", "rb") as f:
    timeTableLinkList = pickle.load(f)

In [ ]:
"""ダイヤの取得"""
for l in tqdm(timeTableLinkList):
    html_text = requests.get(l).text
    soup = BeautifulSoup(html_text, 'html.parser')

    route_name = soup.find("h2",class_="head-txt").get_text()
    route_name = route_name.replace("/","_").replace(":","-")
    operation_name = soup.find("div",class_="head-txt-sub").get_text()
    operation_name = operation_name.replace("/","_").replace(":","-")

    if os.path.isfile(DirTimeSchedule + "/timeTable/"+route_name+"/"+operation_name+".csv"): # 同データが存在する場合は以下処理はスキップ
        continue
    else:
        station_name_list =[]
        d_time_list = []
        for element in soup.find_all("dl",class_="stops"):
            station_name_list.append(element.find("a",class_="station-name-link").text)
        for tElement in soup.find_all("dl",class_="stops"):
            d_time_list.append(tElement.find("dd",class_="time").text[:5])
        os.makedirs("C:/research/バス路線データ/scraping/timeTable/"+route_name+"/", exist_ok=True)
        pd.DataFrame([station_name_list,d_time_list]).T.set_axis([route_name, operation_name], axis=1).to_csv("C:/research/バス路線データ/scraping/timeTable/"+route_name+"/"+operation_name+".csv")